In [1]:
import os

In [2]:
%pwd

'f:\\ProjectAI\\FaceEmotionRecognitionSystem\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\ProjectAI\\FaceEmotionRecognitionSystem'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from FacialExpressionRecognition.constants import *
from FacialExpressionRecognition.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from FacialExpressionRecognition import logger
from FacialExpressionRecognition.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded file from :[{dataset_url}] and saved at :[{zip_download_dir}]")
        except Exception as e:
            raise e

    def extract_zip_file(self):
        unzip_dir = self.config.unzip_dir

        if not os.path.exists(unzip_dir):
            os.makedirs(unzip_dir, exist_ok=True)
            logger.info(f"Extracting zip file :[{self.config.local_data_file}] to dir :[{unzip_dir}]")
            with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
                zip_ref.extractall(unzip_dir)
            logger.info(f"Extracted zip file :[{self.config.local_data_file}] to dir :[{unzip_dir}]")
        else:
            logger.info(f"Zip file already extracted in dir :[{unzip_dir}]")


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-27 18:05:16,751: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-27 18:05:16,753: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-27 18:05:16,754: INFO: common]: Directory created at: artifacts
[2025-09-27 18:05:16,754: INFO: common]: Directory created at: artifacts/data_ingestion


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=10cuSJ1L8N5CT5cmLvieyCf5TFtEz1lLy
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=10cuSJ1L8N5CT5cmLvieyCf5TFtEz1lLy&confirm=t&uuid=164a900a-a828-4417-a0f2-a8602ce907c1
To: f:\ProjectAI\FaceEmotionRecognitionSystem\artifacts\data_ingestion\data.zip
100%|██████████| 40.0M/40.0M [00:00<00:00, 46.9MB/s]

[2025-09-27 18:05:20,454: INFO: 717126125]: Downloaded file from :[https://drive.google.com/file/d/10cuSJ1L8N5CT5cmLvieyCf5TFtEz1lLy/view?usp=sharing] and saved at :[artifacts/data_ingestion/data.zip]
[2025-09-27 18:05:20,455: INFO: 717126125]: Zip file already extracted in dir :[artifacts/data_ingestion]
